In [1]:
from utils import *
import numpy as np
import torch, os
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
set_random_seed(7)

Seed: 7


In [3]:
# def scaling_embeddings(embeddings):
#     for i in range(len(embeddings)):
#         embeddings[i] = StandardScaler().fit_transform(embeddings[i])
#     return embeddings

def reshape_embeddings(embeddings):
    if embeddings.shape[1] == 1:
        embeddings = embeddings.squeeze(1)
    return embeddings

In [4]:
dynahate_labels_train = process_labels(read_labels("dynahate", "train"))
dynahate_labels_dev = process_labels(read_labels("dynahate", "dev"))
dynahate_labels_test = process_labels(read_labels("dynahate", "test"))

latenthatred_labels_train = read_labels("latenthatred", "train")
latenthatred_labels_dev = read_labels("latenthatred", "dev")
latenthatred_labels_test = read_labels("latenthatred", "test")

olid_labels_train = read_labels("olid", "train")
olid_labels_dev = read_labels("olid", "dev")
olid_labels_test = read_labels("olid", "test")

In [5]:
mlp = MLPClassifier(random_state=7)
gridsearch = GridSearchCV(
    mlp,
    param_grid={
        "hidden_layer_sizes": [(128), (128,64)],
        "activation": ["relu"],
        "solver": ["adam"],
        "learning_rate_init": [0.001, 0.0001],
        "learning_rate": ["adaptive"],
        "early_stopping": [True],
        "max_iter": [10000],
    },
    verbose=4,
    n_jobs=os.cpu_count()//3,
    cv=5,
)

In [6]:
dynahate_labels_train_dev = np.concatenate((dynahate_labels_train, dynahate_labels_dev))
latenthatred_labels_train_dev = np.concatenate((latenthatred_labels_train, latenthatred_labels_dev))
olid_labels_train_dev = np.concatenate((olid_labels_train, olid_labels_dev))

In [7]:
bert_dynahate_train_embeddings = reshape_embeddings(bert_dynahate_train_embeddings)
bert_dynahate_dev_embeddings = reshape_embeddings(bert_dynahate_dev_embeddings)
bert_dynahate_test_embeddings = reshape_embeddings(bert_dynahate_test_embeddings)

bert_latenthatred_train_embeddings = reshape_embeddings(bert_latenthatred_train_embeddings)
bert_latenthatred_dev_embeddings = reshape_embeddings(bert_latenthatred_dev_embeddings)
bert_latenthatred_test_embeddings = reshape_embeddings(bert_latenthatred_test_embeddings)

bert_olid_train_embeddings = reshape_embeddings(bert_olid_train_embeddings)
bert_olid_dev_embeddings = reshape_embeddings(bert_olid_dev_embeddings)
bert_olid_test_embeddings = reshape_embeddings(bert_olid_test_embeddings)

### BERT

#### DynaHate

In [8]:
bert_dynahate_train_dev = np.concatenate((bert_dynahate_train_embeddings, bert_dynahate_dev_embeddings))

grid_results = gridsearch.fit(bert_dynahate_train_dev, dynahate_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bert_dynahate_train_dev)
test_preds = mlp.predict(bert_dynahate_test_embeddings)

computeAllScores(train_dev_preds, test_preds, dynahate_labels_train_dev, dynahate_labels_test, "Results/bert_dynahate_seed_7")
save_model(mlp, "Saves/bert_dynahate_seed_7")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.6835296024200519
Accuracy Test:  0.6614077669902912
Weighted F1 Train Dev:  0.678763064879736
Weighted F1 Test:  0.6556062878949919
Macro F1 Train Dev:  0.6747319525009906
Macro F1 Test:  0.6489209269430679
Micro F1 Train Dev:  0.6835296024200519
Micro F1 Test:  0.6614077669902912
Weighted Recall Train Dev:  0.6835296024200519
Weighted Recall Test:  0.6614077669902912
Macro Recall Train Dev:  0.6749692857438782
Macro Recall Test:  0.6489818909725318
Micro Recall Train Dev:  0.6835296024200519
Micro Recall Test:  0.6614077669902912
Confusion Matrix Train Dev: 
[[ 9609  7508]
 [ 4209 15698]]
Confusion Matrix Test: 
[[ 974  878]
 [ 517 1751]]


#### LatentHatred

In [9]:
bert_latenthatred_train_dev = np.concatenate((bert_latenthatred_train_embeddings, bert_latenthatred_dev_embeddings))

grid_results = gridsearch.fit(bert_latenthatred_train_dev, latenthatred_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bert_latenthatred_train_dev)
test_preds = mlp.predict(bert_latenthatred_test_embeddings)

computeAllScores(train_dev_preds, test_preds, latenthatred_labels_train_dev, latenthatred_labels_test, "Results/bert_latenthatred_seed_7")
save_model(mlp, "Saves/bert_latenthatred_seed_7")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': 128, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.6791433891992551
Accuracy Test:  0.680633147113594
Weighted F1 Train Dev:  0.6399694950889232
Weighted F1 Test:  0.6407551175621934
Macro F1 Train Dev:  0.4293201285102874
Macro F1 Test:  0.4283504851503947
Micro F1 Train Dev:  0.6791433891992551
Micro F1 Test:  0.680633147113594
Weighted Recall Train Dev:  0.6791433891992551
Weighted Recall Test:  0.680633147113594
Macro Recall Train Dev:  0.43054539127332087
Macro Recall Test:  0.4287906390185985
Micro Recall Train Dev:  0.6791433891992551
Micro Recall Test:  0.680633147113594
Confusion Matrix Train Dev: 
[[8954  985    6]
 [3365 1968   14]
 [ 457  342   19]]
Confusion Matrix Test: 
[[3021  325    0]
 [1119  627    7]
 [ 147  117    7]]


#### OLID

In [10]:
bert_olid_train_dev = np.concatenate((bert_olid_train_embeddings, bert_olid_dev_embeddings))

grid_results = gridsearch.fit(bert_olid_train_dev, olid_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bert_olid_train_dev)
test_preds = mlp.predict(bert_olid_test_embeddings)

computeAllScores(train_dev_preds, test_preds, olid_labels_train_dev, olid_labels_test, "Results/bert_olid_seed_7")

save_model(mlp, "Saves/bert_olid_seed_7")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': 128, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.754607250755287
Accuracy Test:  0.7872093023255814
Weighted F1 Train Dev:  0.7544593159402319
Weighted F1 Test:  0.7837921371640592
Macro F1 Train Dev:  0.7231816693285464
Macro F1 Test:  0.7273346257997786
Micro F1 Train Dev:  0.754607250755287
Micro F1 Test:  0.7872093023255814
Weighted Recall Train Dev:  0.754607250755287
Weighted Recall Test:  0.7872093023255814
Macro Recall Train Dev:  0.7229149526943645
Macro Recall Test:  0.7209005376344086
Micro Recall Train Dev:  0.754607250755287
Micro Recall Test:  0.7872093023255814
Confusion Matrix Train Dev: 
[[7226 1614]
 [1635 2765]]
Confusion Matrix Test: 
[[540  80]
 [103 137]]
